In [1]:
import pandas as pd
import numpy as np
import os

In [20]:
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm #方差分析
def varianceAna(df_data:pd.DataFrame):
    label=df_data["label"]
    df_data.drop(columns=["label"],axis=1,inplace=True)
    columns_list=list(df_data.columns)
    fea_list=[]
    
    object_string="label~"#识别不了某些特殊字符，先转为fea0，fea1,fean格式
    for index,column in enumerate(columns_list):
        fea_list.append("fea"+str(index))
        object_string=object_string+"fea"+str(index)+'+'
    df_data.columns=fea_list
    df_data['label']=label
    
        
    model=ols(object_string[:-1],df_data).fit()
    anovad = anova_lm(model)
    columns_list.append("Residual")
    anovad.index=columns_list
    columns_list.remove("Residual")
    columns_list.append("label")
    df_data.columns=columns_list
    return anovad

from scipy.stats import f # F检验
def F_test(df_data:pd.DataFrame):
    columns_list=list(df_data.columns)
    columns_list.remove("label")
    p_value_list=[]
    b=df_data["label"]
    for _ in columns_list:
        a=df_data[_]
        F = np.var(a) / np.var(b)
        df1 = len(a) - 1
        df2 = len(b) - 1
        p_value = 1 - 2 * abs(0.5 - f.cdf(F, df1, df2))
        p_value_list.append(p_value)
    return pd.DataFrame({"feature":columns_list,"p_value":p_value_list})

In [3]:
# df_data=pd.DataFrame({"c1":[1,1,3,2,4,1,2,2, 2,2,3,4,3,1,2,1],
#                       "c2":[1,2,3,4,3,1,3,1, 3,2,1,3,4,1,2,3],
#                       "c3":[1,3,4,2,3,3,3,2, 4,2,2,3,3,2,2,1],
#                       "c4":[1,2,2,4,1,3,3,4, 3,2,1,3,2,4,2,3],
#                       "c5":[1,2,6,4,2,6,6,2, 6,1,2,3,6,6,1,2],
#                       "c6":[1,2,3,4,1,4,4,1, 2,3,1,4,4,1,4,1],
#                       "label":[0,0,1,0,0,1,1,0, 1,0,0,0,1,1,0,0]})
# varianceAna(df_data)

In [28]:
def extract_paras_columns(excel_filename,sheet_name,file_root="F:\\YIELD\\youdaOptronics\\Archive(2)",csv_generate_flag=False):
    rawData=pd.read_excel(os.path.join(file_root,excel_filename),sheet_name=sheet_name)
    columns_list=list(rawData.columns)
    para_columns_list=[]
    for column in columns_list:
        if 'TIME' not in column:
            para_columns_list.append(column)
    df_para=rawData[para_columns_list].copy()
    df_para['label']=0
    df_para['label'][df_para['Y']>0]=1
#     print(df_para.info())
    if(csv_generate_flag):
        df_M3_CLA_para.to_csv(os.path.join(file_root,"data_generate",excel_filename[:-5]+"_paras.csv"),index=False)
    return df_para

In [29]:
df_M3_CLA=extract_paras_columns("M3-CLA.xlsx",'1_1')
df_M3_CLA.drop(columns=['SHEET_ID','Y','OPERATION_ID','TOOL_ID','ABBR_NO','RECIPE_ID'],inplace=True)

E:\anaconda\install\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


In [30]:
result_df=varianceAna(df_M3_CLA)
result_df.sort_values(by=["PR(>F)"])

,df,sum_sq,mean_sq,F,PR(>F)
AK_Process_QUANTITY,1.0,2.753896,2.753896,10.120567,0.001789
EQ_Process_QUANTITY,1.0,1.086507,1.086507,3.992912,0.047536
HPMJ/FS_Process_QUANTITY,1.0,1.052206,1.052206,3.866858,0.051132
SH/KJ_Process_QUANTITY,1.0,0.966355,0.966355,3.551354,0.061471
PANEL_LENGTH,1.0,0.885260,0.885260,3.253329,0.073326
IN_CV_HEPA_Run (0:OFF//1:ON),1.0,0.480707,0.480707,1.766599,0.185863
PANEL_THICKNESS,1.0,0.467343,0.467343,1.717487,0.192061
IN_CV_SPEED,1.0,0.449235,0.449235,1.650938,0.200852
Out_CV_SPEED,1.0,0.382716,0.382716,1.406482,0.237555
Out_CV_ESD_MONITOR,1.0,0.347398,0.347398,1.276687,0.260356


In [26]:
result_df_f_test=F_test(df_M3_CLA)
result_df_f_test.sort_values(by=["p_value"])

,feature,p_value
16,PROCESS_CV_SPEED,0.000000e+00
2,Glass Data,0.000000e+00
15,PANEL_WIDTH_,0.000000e+00
4,IN_CV_HEPA_Run (0:OFF//1:ON),0.000000e+00
5,IN_CV_ION_BAR_Run (0:OFF//1:ON),0.000000e+00
14,PANEL_THICKNESS,0.000000e+00
7,NE_HEPA_Run (0:OFF//1:ON),0.000000e+00
13,PANEL_LENGTH,0.000000e+00
12,Out_CV_SPEED,0.000000e+00
10,Out_CV_HEPA_Run (0:OFF//1:ON),0.000000e+00
